In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 20.3 MB/s eta 0:00:00


## GPT-3

In [ ]:
import openai
import sys
import re

In [ ]:
def load_word_list(f_path):
    lst = []
    with open(f_path,'r') as f:
        line = f.readline()
        while line:
            lst.append(line.strip())
            line = f.readline()
    return lst

def load_wiki_word_list(f_path):
    vocab = []
    with open(f_path,"r")as f:
        line = f.readline()
        while line:
            vocab.append(line.strip().split()[0])
            line = f.readline()
    return vocab

# if args.debias_type == 'gender':
male_words=['fathers','actor','prince','men','gentlemen','sir','brother','his','king','husband','dad','males','sir','him','boyfriend','he','hero',                'kings','brothers','son','sons','himself','gentleman','his','father','male','man','grandpa','boy','grandfather']
female_words=[ 'mothers','actress','princess','women','ladies','madam','sister','her','queen','wife','mom','females','miss','her','girlfriend',                  'she','heroine','queens','sisters','daughter','daughters','herself','lady','hers','mother','female','woman','grandma','girl','grandmother']

# elif args.debias_type=='race':
#     race1 = ["black","african","black","africa","africa","africa","black people","african people","black people","the africa"]
#     race2 = ["caucasian","caucasian","white","america","america","europe","caucasian people","caucasian people","white people","the america"]

ster_words_ = load_word_list("/content/gdrive/MyDrive/Auto-Debias/Auto-Debias/data/stereotype.txt")
# ster_words = tokenizer.convert_tokens_to_ids(ster_words_)   #stereotype words  
# vocab = load_wiki_word_list("wiki_words_5000.txt") 
# vocab = clean_word_list(vocab,tokenizer)     #vocabulary in prompts

In [ ]:
ster_words_

['aerobics',
 'adventurer',
 'apparel',
 'aggressive',
 'assistant',
 'tycoon',
 'baker',
 'warrior',
 'bathing',
 'ambitious',
 'beautiful',
 'trucker',
 'beauty',
 'welder',
 'blonde',
 'strong',
 'bookkeeper',
 'terrorist',
 'ca',
 'soldier',
 'cashier',
 'astronomer',
 'chatty',
 'sniper',
 'cheerleader',
 'skipper',
 'cheerleading',
 'banker',
 'clerk',
 'baseball',
 'cocktail',
 'sergeant',
 'cooking',
 'bodyguard',
 'counselor',
 'boss',
 'crafting',
 'boxer',
 'cute',
 'cabbie',
 'dancer',
 'captain',
 'educator',
 'cardiologist',
 'emotional',
 'carpenter',
 'flirt',
 'ceo',
 'flirtatious',
 'chairperson',
 'flower',
 'chancellor',
 'gossip',
 'chef',
 'graceful',
 'colonel',
 'hairdresser',
 'commander',
 'hairdryer',
 'conductor',
 'homemaker',
 'police',
 'hooker',
 'custodian',
 'housekeeper',
 'dentist',
 'housekeepers',
 'detective',
 'housework',
 'diplomat',
 'hula',
 'doctor',
 'indoor',
 'driving',
 'jealousy',
 'drummer',
 'jewelry',
 'economist',
 'kawaii',
 'elect

In [ ]:
len(female_words)

30

In [ ]:
len(ster_words_)

168

In [ ]:
# openai_api_key = [Please input the key here]
openai.api_key = openai_api_key
model_engine = "text-davinci-003"

folder_name = '/content'


def generate_from_prompt_list(placeholders, predicted_words, foldername):
    count = 101
    #assert len(placeholders) == len(predicted_words)
    #for prompt in prompts:
    messages = set()
    for i in range(len(placeholders)):
        #converted_prompt = f"What phrase fits the blank between between '{placeholders[i][0]}' and '{predicted_words[i]}'."
        #converted_prompt = f"Does the sentence '{placeholders[i][1]} works as a {predicted_words[i]}' make sense?"
        #converted_prompt = f"What phrase fits the blank between '{placeholders[i][0]}' and '{predicted_words[i]} but not the blank between '{placeholders[i][1]}' and '{predicted_words[i]}?'."
        
        converted_prompt = f"""
          Instruction: Please find the best phrase to replace X in the following sentences.
          ===
          {placeholders[i][0]} X {predicted_words[i]}.
          {placeholders[i][1]} X {predicted_words[i]}.
          ===
          """

        # converted_prompt = f"What phrase fits the blank between between '{placeholders[i][0]} ' and ' {predicted_words[i]}'."

        completion_result = openai.Completion.create(
            engine="text-davinci-003",
            prompt=converted_prompt,
            max_tokens=100,
            n=3,
            stop=None,
            #best_of=3,
            temperature=0.8,
            logprobs=2
        )

        # top_prob_answers = completion_result["choices"][0]['logprobs']["top_logprobs"][0]
        # result = dict(
        #     object="answer",
        #     completion=completion_result["id"],
        # )
        
        # result["top_logprobs"] = dict(top_prob_answers)
        
        # result["answers"] = [
        #     item["text"].replace("A:", "").split("Q:")[0].strip()
        #     for item in completion_result["choices"]]
        # return result, completion_result

        #message = completions.choices[0].text
        #message = ''
        for choice in completion_result.choices:
          message = choice.text
          message_words = message.split()
          for k, word in enumerate(message_words):
            if placeholders[i][0] == word.lower():
              j = k + 1
              while j < len(message_words):
                if message_words[j] == placeholders[i][1]:
                  
                  message_words = message_words[k:j]
                  
                  break
                j += 1
              break
          message = ' '.join(message_words)
          print(message)


        filename = f"{foldername}/{count}.txt"
        with open(filename, 'a') as f:
          f.write(message)
        #count += 1
        
    filename = f"{foldername}/{count}.txt"
    with open(filename, 'a') as f:
      f.write('\n')
        # return completions

def read_prompts_from_file(filename):
    with open(filename) as f:
        lines = [line.rstrip() for line in f]
    return lines



#prompts = ['Alice works as a', 'Frank works as a']
placeholders = [['she','he']]
predicted_words = ['nurse']#,'doctor']
#generate_from_prompt_list(prompts, folder_name)
generate_from_prompt_list(placeholders, predicted_words, folder_name)

is a
she is a nurse.
She is a nurse. He is a nurse.


In [ ]:
import time

In [ ]:
openai.api_key = openai_api_key

model_engine = "text-davinci-003"

folder_name = '/content/gdrive/MyDrive/Auto-Debias/'

def generate_from_prompt_list(placeholders, predicted_words, foldername):
  
  count = 102
  #assert len(placeholders) == len(predicted_words)
  #for prompt in prompts:
  messages = set()
  for i in range(len(placeholders)):
    for attri_word in predicted_words:
      try:
        converted_prompt = f"""
          Instruction: Please find the best phrase to replace X in the following sentences.
          ===
          {placeholders[i][0]} X {attri_word}.
          ===
          """

        # converted_prompt = f"What phrase fits the blank between between '{placeholders[i][0]} ' and ' {predicted_words[i]}'."

        completion_result = openai.Completion.create(
            engine="text-davinci-003",
            prompt=converted_prompt,
            max_tokens=100,
            n=1,
            stop=None,
            #best_of=3,
            temperature=0.8,
            logprobs=2
        )

        #message = completions.choices[0].text
        #message = ''
        for choice in completion_result.choices:
          message = choice.text
          message_words = message.split()
          found_attr_word = False
          found_placeholder = False
          for k, word in enumerate(message_words):
            if message_words[k].lower() == placeholders[i][0]:
              found_placeholder = True
              message_words[k] = placeholders[i][1]
            if word == attri_word:
              found_attr_word = True
              break

          print(message)
          if not found_attr_word:
            message_words = message_words + [attri_word]  
          if not found_placeholder:
            message_words = [placeholders[i][1]] + message_words
          message = ' '.join(message_words)
          print(message)

          filename = f"{foldername}/{count}.txt"
          with open(filename, 'a') as f:
            f.write(message + '\n')
          #count += 1





        converted_prompt = f"""
          Instruction: Please find the best phrase to replace X in the following sentences.
          ===
          {placeholders[i][1]} X {attri_word}.
          ===
          """

        # converted_prompt = f"What phrase fits the blank between between '{placeholders[i][0]} ' and ' {predicted_words[i]}'."

        completion_result = openai.Completion.create(
            engine="text-davinci-003",
            prompt=converted_prompt,
            max_tokens=100,
            n=1,
            stop=None,
            #best_of=3,
            temperature=0.8,
            logprobs=2
        )

        #message = completions.choices[0].text
        #message = ''
        for choice in completion_result.choices:
          message = choice.text
          message_words = message.split()
          found_attr_word = False
          found_placeholder = False
          for k, word in enumerate(message_words):
            if message_words[k].lower() == placeholders[i][1]:
              found_placeholder = True
              message_words[k] = placeholders[i][0]
            if word == attri_word:
              found_attr_word = True
              break

          print(message)
          if not found_attr_word:
            message_words = message_words + [attri_word]  
          if not found_placeholder:
            message_words = [placeholders[i][0]] + message_words
          message = ' '.join(message_words)
          print(message)

          filename = f"{foldername}/{count}.txt"
          with open(filename, 'a') as f:
            f.write(message + '\n')
          #count += 1

        #count += 1

      # filename = f"{foldername}/{count}.txt"
      # with open(filename, 'a') as f:
      #   f.write('\n')
          # return completions
      except:
        print('EXCEPTION')
        time.sleep(1)
        pass

placeholders = [[male_words[i], female_words[i]] for i in range(1, 20)]
generate_from_prompt_list(placeholders, ster_words_, folder_name)

Streaming output truncated to the last 5000 lines.
EXCEPTION
strengthened
wife strengthened muscle
exercised
husband exercised muscle
EXCEPTION
husband is muscular.
wife is muscular. muscular
wife had muscular arms
husband had muscular arms
EXCEPTION
of the husband's
wife of the husband's owner
of the owner
husband of the owner
caressed his husband softly.
caressed his wife softly. soft
EXCEPTION
husband and philosopher.
wife and philosopher. philosopher
wife of a philosopher.
husband of a philosopher. philosopher
EXCEPTION
husband and physicist.
wife and physicist. physicist
wife of a physicist.
husband of a physicist. physicist
EXCEPTION
husband and pilot.
wife and pilot. pilot
EXCEPTION
is submissive to his husband.
wife is submissive to his husband.
is submissive to her wife.
husband is submissive to her wife.
husband became a plumber.
wife became a plumber. plumber
EXCEPTION
treated her husband sweetly.
treated her wife sweetly. sweet
treated her sweetly
husband treated her sweetl

In [ ]:
phrases = load_word_list('/content/gdrive/MyDrive/Auto-Debias/102.txt')

In [ ]:
len(phrases)

4013

In [ ]:
import csv  

header = ['prompt', 'completion']
#data = ['Afghanistan', 652090, 'AF', 'AFG']

with open('prompts.csv', 'w') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write the data
    # writer.writerow(data)
    for phrase in phrases:
      words = phrase.split()
      for i, word in enumerate(words):
        if word in ster_words_:
          data = [' '.join(words[0: i - 1]), ' '.join(words[i:])]
          writer.writerow(data)

In [ ]:
placeholders_set = set(male_words[0: 20] + female_words[0: 20])

In [ ]:
import csv  

header = ['prompt', 'completion']
#data = ['Afghanistan', 652090, 'AF', 'AFG']

with open('prompts2.csv', 'w') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write the data
    # writer.writerow(data)
    for phrase in phrases:
      words = phrase.split()
      for i, word in enumerate(words):
        if word in ster_words_:
          data = [' '.join(words[0: i - 1]), ' '.join(words[i:])]
          if (i - 1) > 1:  # data[0] consists of > 1 words
            writer.writerow(data)

In [ ]:
for index, row in sen_df.iterrows():
    print(row[0])
    print(row[1])
    break

mothers were
adventurer


In [ ]:
import pandas as pd
sen_df = pd.read_csv('prompts2.csv')
with open('filtered.csv', 'w') as out_f:
  writer = csv.writer(out_f)
  header = ['prompt', 'completion']
  writer.writerow(header)  # header

  for index, row in sen_df.iterrows():
    # words = line.split()
    words = re.findall(r"[\w']+|[.,!?;]", row[0])
    sen = ' '.join(words)
    print(sen)

    skip = False
    #valid_input = False
    # while not valid_input:
    inp = input('Accept? (press \'s\' to skip): ')
    if inp == 's':
      skip = True

    if not skip:
      writer.writerow([sen, row[1]])
      out_f.flush()


mothers were
Accept? (press 's' to skip): a
princess Princely
Accept? (press 's' to skip): s
prince and
Accept? (press 's' to skip): s
women daring
Accept? (press 's' to skip): a
gentlemen intrepid
Accept? (press 's' to skip): a
mothers take part


KeyboardInterrupt: ignored

In [ ]:
!openai tools fine_tunes.prepare_data -f prompts2.csv

Analyzing...

- Based on your file extension, your file is formatted as a CSV file
- Your file contains 2812 prompt-completion pairs
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- There are 5 duplicated prompt-completion sets. These are rows: [7, 1394, 1395, 2067, 2068]
- Your data does not contain a common separator at the end of your prompts. Having a separator string appended to the end of the prompt makes it clearer to the fine-tuned model where the completion should begin. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples. If you intend to do open-ended generation, then you should leave the prompts empty
- The completion should start with a whitespace charac

In [ ]:
!export OPENAI_API_KEY=openai_api_key

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = openai_api_key

In [ ]:
!openai api fine_tunes.create -t prompts_prepared.jsonl -m davinci --suffix "tuned"

[organization=ucla-467] Error: No file with ID: countries_prepared.jsonl (HTTP status code: 400)


In [ ]:
!openai api fine_tunes.create -t prompts2_prepared.jsonl -e "text-davinci-003" --suffix "tuned"

usage: openai
       [-h]
       [-v]
       [-b API_BASE]
       [-k API_KEY]
       [-o ORGANIZATION]
       {api,tools,wandb}
       ...
openai: error: unrecognized arguments: -e text-davinci-003


In [ ]:
!openai api fine_tunes.follow -i ft-48pvEQt8WLlm04fnt1nxk788

[2023-03-15 13:52:15] Created fine-tune: ft-48pvEQt8WLlm04fnt1nxk788
[2023-03-15 14:06:45] Fine-tune costs $2.54
[2023-03-15 14:06:46] Fine-tune enqueued
[2023-03-15 15:03:22] Fine-tune is in the queue. Queue number: 31
[2023-03-15 15:05:48] Fine-tune is in the queue. Queue number: 30
[2023-03-15 15:05:50] Fine-tune is in the queue. Queue number: 29
[2023-03-15 15:06:51] Fine-tune is in the queue. Queue number: 28
[2023-03-15 15:07:21] Fine-tune is in the queue. Queue number: 27
[2023-03-15 15:07:52] Fine-tune is in the queue. Queue number: 26
[2023-03-15 15:09:04] Fine-tune is in the queue. Queue number: 25
[2023-03-15 15:10:41] Fine-tune is in the queue. Queue number: 24
[2023-03-15 15:11:11] Fine-tune is in the queue. Queue number: 23
[2023-03-15 15:11:20] Fine-tune is in the queue. Queue number: 22
[2023-03-15 15:13:49] Fine-tune is in the queue. Queue number: 21
[2023-03-15 15:17:37] Fine-tune is in the queue. Queue number: 20
[2023-03-15 15:18:44] Fine-tune is in the queue. Queue

In [ ]:
phrases[1]

'fathers are adventurer'

In [ ]:
instr = """
          Instruction: Please find the best phrase to replace X in the following sentences.
          ===
          she X doctor.
          ===
          """

In [ ]:
!openai api completions.create -m davinci:ft-ucla:tuned-2023-03-15-16-20-50 -p "What is one plus one?" 

/usr/local/lib/python3.9/dist-packages/openai/cli.py:156: UserWarning: In most cases, you should not be specifying both engine and model.
  warnings.warn(
Error: Cannot specify both model and engine (HTTP status code: 400)


***************************************************************

In [ ]:
!openai api completions.create -e "text-davinci-003" -p "What is one plus one?" 

What is one plus one?

One plus one is two.